In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import base64
import pandas as pd
import plotly.express as px
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Database connection setup
username = "aacuser"
password = "SNHU1234"
db = AnimalShelter(
    user=username, 
    password=password, 
    host='nv-desktop-services.apporto.com', 
    port=33280, 
    db='AAC', 
    collection='animals'
)

# Initial data retrieval
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Load and encode the Grazioso Salvare logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Dashboard layout
app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.Img(
        src='data:image/png;base64,{}'.format(encoded_image.decode()), 
        style={'width': '200px', 'margin-bottom': '20px'}
    )),
    html.Center(html.P("Developed by Trace Bailey", style={'font-weight': 'bold', 'font-size': '16px'})),
    html.Hr(),
    html.Div([
        # Dropdown for animal type
        dcc.Dropdown(
            id='filter-animal-type',
            options=[
                {'label': 'Dog', 'value': 'Dog'},
                {'label': 'Cat', 'value': 'Cat'},
                {'label': 'Other', 'value': 'Other'}
            ],
            placeholder="Select Animal Type",
            style={'width': '40%', 'margin': '10px auto'}
        ),
        # Dropdown for outcome type
        dcc.Dropdown(
            id='filter-outcome-type',
            options=[
                {'label': 'Adoption', 'value': 'Adoption'},
                {'label': 'Euthanasia', 'value': 'Euthanasia'},
                {'label': 'Return to Owner', 'value': 'Return to Owner'},
                {'label': 'Transfer', 'value': 'Transfer'},
            ],
            placeholder="Select Outcome Type",
            style={'width': '40%', 'margin': '10px auto'}
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'},
        style_cell={'textAlign': 'left'},
        row_selectable='single'  # Allow single row selection
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-animal-type', 'value'),
               Input('filter-outcome-type', 'value')])
def update_dashboard(animal_type, outcome_type):
    query = {}
    if animal_type:
        query['animal_type'] = animal_type
    if outcome_type:
        query['outcome_type'] = outcome_type
    filtered_data = db.read(query)
    return pd.DataFrame.from_records(filtered_data).drop(columns=['_id']).to_dict('records')

@app.callback(Output('graph-id', "children"),
              [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData:
        dff = pd.DataFrame.from_dict(viewData)
        return [
            dcc.Graph(
                figure=px.pie(dff, names='breed', title='Preferred Breeds')
            )
        ]
    return []

@app.callback(Output('map-id', "children"),
              [Input('datatable-id', "derived_virtual_data"),
               Input('datatable-id', "selected_rows")])
def update_map(viewData, selected_rows):
    if viewData and selected_rows:
        dff = pd.DataFrame.from_dict(viewData)
        row = selected_rows[0]  # Get the index of the selected row
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[
                dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']
            ], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], children=[
                    dl.Tooltip(dff.iloc[row]['breed']),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[row]['name'])
                    ])
                ])
            ])
        ]
    return []

# Run the app
app.run_server(debug=True)

Dash app running on http://127.0.0.1:10664/
